<a href="https://colab.research.google.com/github/IreneYIN7/CSC-Data_Mining-Machine_Learning_Projects/blob/master/Irene_CSC_321_Assignment_3_MLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC-321: Data Mining and Machine Learning

# Irene Yin

## Assignment 3: Multivariate Linear Regression


The function for making predicitions using Simple Linear Regression was:

**y = b0 + b1*x**

For the more complicated Multivariate Linear Regression the function is:

**y = b0 + b1*x1 + b2*x2 + ... + bN*xN**

With there being as many coefficients (b) as there are input features, plus 1 (for the b0 coefficient, or intercept). 

For more complex learning, we cannot simply read our coefficients (b0 and b1 for SLR) from the data. We *could* use the least squares method, but for MORE input coefficients we have to use more complex linear algebra that involves lots of matrix math. And in the end it will be an *estimation* of the coefficients. 

We have to estimate them because there are many more of them, and they all interact together. Least squares is one method of estimation. We're going to look at another, Stochastic Gradient Descent (SGD).

We don't know exactly how the input variables play together for a given data set so we're going to use a mechanism to pick some random values for our initial coefficients, and gradually improve the coefficients over time.

The method we discussed in class is called Stochastic Gradient Descent, and is one of a variety of optimization algorithms that are used in machine learning methods to 'learn' the coefficients, or weights, on input values with respect to some output. This estimation of coefficients IS the learning part of machine learning. At least for these linear models.

Gradient descent is the process of minimizing some function (we'll call it the cost, or error function) by following the slope (or gradient) of the function down to some minimum (the lowest error over our data). 

Intuitively, we're going to show our model one training instance at a time, make a prediction for that instance, calculate the error using that instance, and update the model in order to improve performance (get a smaller error) for the next prediction. We'll repeat this process for a number of iterations, minimizing the error each time.

Each iteration, we're going to update each of the coefficients using the formula:

b = b - learning_rate * error * value_of_x

for each coefficient (again, corresponding to each input feature, of each instance). The error means the error calculated when we use the coefficients to make a prediction. So for coefficient b1, this translates to:

b1 = b1 - learning_rate * error * value_of_x1

Remember also that learning_rate is a value we must choose (I'll tell you to start with), and the total number of iterations over all the training data (epochs) is ALSO a number we must choose. 

We're going to break this down into the individual steps. I recommend reading this whole notebook to get a sense of where we're going.

Let's start.

### Part 1: Making Predictions

(a) We're going to use predictions made by our model as a guide for tuning the coefficients, so we need a function that can make predictions. This *same* function can also apply the final coefficients we have learned to make predictions over new data. 

Write a function called that takes a **single instance** (a list of input features, from for example an X_train or X_test data set), and a list of coefficients, and calculates the predicted y value, using the formula:

y = b0 + b1*x1 + b2*x2 + ... + bN*xN

This should work for ANY length of instance, but we can always assume that the length of the instance list and the length of the coefficent list are different by one. Why one? For the instance, imagine that the input feature values of a single instance are:

[x1,x2,x3] 

For the coefficients, let's assume that the list contains all the coefficients, including b0. Let's also assume that we ALWAYS store the b0 coefficient at position coefficients[0] (i.e. the first position in the coefficients list).

The resulting coefficient list for the above instance would then be:

[b0,b1,b2,b3]

Again, your code should work for ANY length of instance, and the corresponding list of coefficients.

Your function should return the predicted y value for a given instance and a given set of coefficients, using the formula above. 

(b) In the Simple Linear Regression assignment, you applied your model to a contrived data set. I've reproduced this data set below. Go through this data set one instance at a time and call your new predict function for each instance. You can use the coefficients [0.4, 0.8], which are almost exactly what you learned as coefficients in Assignment 2. 

For each instance, print out the correct value and the value predicted by your function from (a). You should see that it performs reasonably well - the predicted values should be close to but not exactly the same as the actual values.

I'd take the time to print nicely here. It helps.

In [ ]:
import numpy as np

# Write your predict function here

def predFunc(inputlist, coefficient):
  # input: the length of coeficent is one more than the length of inputlist. 
  #        We assume the coefficient list contains all the coefficients, including b0.
  # return: the predicted output value.
  predOutput = coefficient[0]
  for i in range(len(coefficient)-1):
    nextIndex = i+1
    predOutput += coefficient[nextIndex]*inputlist[i]
  return predOutput

# Apply your function to the contrived dataset

# Split your data into X_train and y_train
# In this assignment, it's important that X_train be a list of lists, i.e.
# [[1],[2],[4],[3],[5]]
# in this case
# Go through each instance of X_train, and get a predicted y value
# Print out the predicted y, and the corresponding actual y from y_train

dataset = [[1,1],[2,3],[4,3],[3,2],[5,5]]
coef = [0.4,0.8]
X_train = []
for i in range(len(dataset)):
  X_train.append(dataset[i][:len(dataset[i])-1])

y_train = []
for i in range(len(dataset)):
  y_train.append(dataset[i][-1])

for i in range(len(X_train)):
  output = predFunc(X_train[i], coef) 
  print("Predicted y value: %.3f" % output, "Real y value: ", y_train[i] )







Predicted y value: 1.200 Real y value:  1
Predicted y value: 2.000 Real y value:  3
Predicted y value: 3.600 Real y value:  3
Predicted y value: 2.800 Real y value:  2
Predicted y value: 4.400 Real y value:  5


### Part 2: Learning coefficients with Stochastic Gradient Descent

We now need to estimate coefficients for ourselves by writing a stochastic gradient descent function. For a given learning rate, for a number of iterations (epochs), we're going to estimate our coefficients.  Given a set of training data, spilt into X_train and y_train, we're going to:

- loop over each epoch (where we go through ALL the instances of the training data)
- loop over each row (instance) in the training data, for an epoch
- loop over each coefficient, for each feature in each instance, for each epoch

As computer scientists you should recognize that this requires three, nested for loops, and you should have a sense (a big-O kind of sense) why this can take a long time for large data sets.

Coefficients are updated based on the error the model made. The error is calculated as the difference between the predicted y value and the actual y value:

error = prediction - actual

There is one coefficient for EACH input attribute, and these are updated every time, for example:

b1 = b1 - learning_rate * error * x1

We ALSO need to update the special intercept coefficient b0:

b0 = b0 - learning_rate * error

(c) Implement the following algorithm for Stochastic Gradient Descent, that takes a list of X_train values, the list of corresponding y_train values, and two other parameters, learning_rate and epochs.

The algorithm is as follows: 

- initialize a list for the output coefficients. The length of the list will be the same as the length of each instance in your X_train data, + 1 for the special b0 coefficient. We can initialize all the coefficients to 0.0 to start with (remember, it doesn't really matter where we start)
- for each epoch
    - initialize the total error to 0
    - for each instance in the X_train data
        1. calculate the prediction for that instance, given the current list of coefficients, using our function from (a)
        2. calculate the error for that prediction, using the corresponding y_train value 
        3. square the error, and add it to the total error. Note that squaring the individual error means it will always be a positive value. ALSO NOTE: We don't use this squared error for updating the coefficients - we use the error calculated in **step 2**. This squaring is just to give us nice, readable output at the end of each epoch. 
        4. Now update the coefficients, using the formulas given above. One update for b0 (which should always be at position 0 in the coefficients list), and then loop over the remaining coefficients, performing an update for each one, b1 through bN.
        
    - At the end of each epoch, print out the epoch number (we can start at epoch 0), the learning rate, and the total error for this epoch (DO THIS ON ONE LINE FOR READABILITY).
    - repeat for the next epoch
- once we've iterated through all the epochs, return the list of coefficients

(d) Apply your stochastic gradient descent function to the contrived dataset, given below. If it's working, you should see the error rate falling each epoch. You should also note that the value of the coefficients learned at the end isn't quite the same as Simple Linear Regression, because we're estimating this time. You could try learning longer (more epochs), or altering the learning rate, and see if the coefficients approach the optimal values we learned in Assignment 2.

I've given you TWO datasets to work with. The first aligns with last week, so you can test your results. The second involves more features, so you can make sure you haven't accidentally made coding decisions that rely on only two features.

In [ ]:
# Write your coefficientsSGD(train,test,learning_rate,epochs) here
def coefficientsSGD(train,test,learning_rate,epochs):
  # return: the coefficients
  coefficient = [0.0 for j in range(len(train[0])+1)]
  for iteration in range(epochs):
    Totalerror = 0
    for i in range(len(train)):
      prediction = predFunc(train[i],coefficient)
      error = prediction - test[i]
      Totalerror += error**2
      coefficient[0] -= learning_rate * error
      for index in range(len(coefficient)-1):
        coefficient[index+1] -= learning_rate * error * train[i][index]
    print("epoch number: ", iteration, "learning rate is: ", learning_rate,
          "total error is: ", Totalerror)
  return coefficient

# Apply to the contrived data here. Try my parameters first, before you experiment
# BEFORE you submit your assigment, return the code to using my parameters:
# learning_rate = 0.001
# epochs = 50

dataset1 = [[1,1],[2,3],[4,3],[3,2],[5,5]]
dataset2 = [[1,2,1],[2,2,3],[4,3,3],[3,2,2],[5,4,5],[5,2,6],[4,5,4],[7,5,5]]
X_train1 = []
for i in range(len(dataset1)):
  X_train1.append(dataset1[i][:len(dataset1[i])-1])
print(X_train1)
print(len(X_train1[0]))
y_train1 = []
for i in range(len(dataset1)):
  y_train1.append(dataset1[i][-1])

X_train2 = []
for i in range(len(dataset2)):
  X_train2.append(dataset2[i][:len(dataset2[i])-1])

y_train2 = []
for i in range(len(dataset2)):
  y_train2.append(dataset2[i][-1])


learning_rate = 0.001
epochs = 50

coefficient1 = coefficientsSGD(X_train1, y_train1, learning_rate, epochs)
print('The coefficients of the dataset1 is: ', coefficient1)

coefficient2 = coefficientsSGD(X_train2, y_train2, learning_rate, epochs)
print('The coefficients of the dataset2 is: ', coefficient2)
# Slice your data into X_train and y_train
# Call your SGD function to obtain a list of coefficients
# PRINT your coefficients nicely





[[1], [2], [4], [3], [5]]
1
epoch number:  0 learning rate is:  0.001 total error is:  46.23569225016471
epoch number:  1 learning rate is:  0.001 total error is:  41.305142323835085
epoch number:  2 learning rate is:  0.001 total error is:  36.92968879875065
epoch number:  3 learning rate is:  0.001 total error is:  33.046843407651664
epoch number:  4 learning rate is:  0.001 total error is:  29.601151923716085
epoch number:  5 learning rate is:  0.001 total error is:  26.543402390484545
epoch number:  6 learning rate is:  0.001 total error is:  23.82992247422831
epoch number:  7 learning rate is:  0.001 total error is:  21.421955907121237
epoch number:  8 learning rate is:  0.001 total error is:  19.285109118735654
epoch number:  9 learning rate is:  0.001 total error is:  17.38886015544335
epoch number:  10 learning rate is:  0.001 total error is:  15.706122876572774
epoch number:  11 learning rate is:  0.001 total error is:  14.212860205347214
epoch number:  12 learning rate is:  0

(e) Now you have sufficient functionality to write a function to learn and make predictions using multivariate linear regression. 

Create a function that takes:

1. X_train
2. y_train

which are the training data, from which we'll estimate coefficients

3. X_test

our test data, which we'll use to evaluate the model

4. Learning rate
5. Epochs

and our parameters for learning.

Just like the last assignment, we're going to use the same dataset here for both training and testing, even though that might not be a great idea (why not?).

Here's the multivariate linear regression algorithm. We're going to estimate our coefficients from the training data, using the function from (c) above. We're going to create a new list, to hold our predictions. Then for each entry in the testing data, we're going to read the input value, and make a prediction, using our function from (a). We'll append our predicted y value to the prediction list. We're going to return our list of predictions.

(f) Compare the predicitons to the actual y values stored in y_test, and compute the RMSE value. Print it nicely.
Also print the zeroR value. This means you're going to need your RMSE function and your zeroRR function from the previous assignment.

In [ ]:
# Write your function for multivariate linear regression here
X_test = X_train
def mlr(X_train, y_train, X_test, Learning_rate, Epochs):
  coefficient = coefficientsSGD(X_train, y_train, Learning_rate, Epochs)
  predictions = []
  for instance in X_test:
    predictions.append(predFunc(instance, coefficient))
  return predictions

#f)

def rmse(actual, predicted):
  # return: prediction error.
  predictionError = 0.0
  for i in range(len(actual)):
    predictionError += (actual[i] - predicted[i])**2
  predictionError_avg = predictionError/len(actual)
  predictionError_avg = predictionError_avg**0.5
  return predictionError_avg

def zeroRR(ytrain, Xtest):
  # return: compute the mean from the ytrain values and return the list of predictions.
  meanOfytrain = np.mean(ytrain)
  prediction = []
  for num in Xtest:
    prediction.append(meanOfytrain)
  return prediction

predictions = mlr(X_train, y_train, X_test, learning_rate, epochs)
predictions2 = zeroRR(y_train, X_test)
print("The RMSE value is: ", rmse(y_train, predictions))
print("The ZeroR value is: ", rmse(y_train, predictions2))

prediction:  0.0
prediction:  0.003
prediction:  0.031973
prediction:  0.063563351
prediction:  0.132278553384
epoch number:  0 learning rate is:  0.001 total error is:  46.23569225016471
prediction:  0.062783210275696
prediction:  0.11460888582494891
prediction:  0.24047988979213297
prediction:  0.22063162702668088
prediction:  0.3826217302722257
epoch number:  1 learning rate is:  0.001 total error is:  41.305142323835085
prediction:  0.12193316085896903
prediction:  0.21975246248293684
prediction:  0.4369010262119619
prediction:  0.3685976498504865
prediction:  0.6184522052846227
epoch number:  2 learning rate is:  0.001 total error is:  36.92968879875065
prediction:  0.1776604281876328
prediction:  0.3188054518429674
prediction:  0.6219368917998004
prediction:  0.5079889785983791
prediction:  0.8406111270224543
epoch number:  3 learning rate is:  0.001 total error is:  33.046843407651664
prediction:  0.2301633838402014
prediction:  0.41212085715859803
prediction:  0.796247369616324

### Part 3: Introduction to scikit-learn


As we go through this class I'll introduce you to some of the functionality of scikit learn. 

Below I import the Stochastic Gradient Descent regressor. This learns the coefficients using SGD, and then when predict is called uses those coefficients in the multivariate linear regression formula.

Note that when I create an instance of the SGDRegressor, I give a parameter which is the MAXIMUM number of iterations before stopping. The implementation of SGD in scikit learn is a little more advanced than what you did above, in that it will AUTOMATICALLY stop when the amount of learning (the change in error) stops changing significantly enough. 

I've added some code to print out the intercept (b0), the list of input coefficients, AND the number of iterations before it stopped. 

Recreate the four lists X_train, y_train, X_test and y_test as you would have for the previous examples, using the same data for both training and testing. Then the following code should run. 

Note that I have to do some reshaping, to make them into 2D arrays from the 1D list that python uses. This is just a requirement of scikit-learn, and you can see how I do it below.

You can find out more in general at: https://scikit-learn.org/stable/index.html

This time however, I want you to add the following:
- Add in the linear regression from sklearn from Assignment 2(* see the note below)
- Add in the dummy regressor (zeroR)
- Add in RMSE evaluation for all three models (SGDRegressor, Linear Regression and zeroR)
- Print out the RMSE for all three
- Add a text box AFTER the results commenting on which model(s) are better and why
- As a last thing, CHANGE the max iterations in the SGD regressor to 20. You SHOULD get a warning when you run it about the model NOT converging. This means it did NOT have enough 'time' to find the best solution.

Think carefully about structuring your code, and presenting the output. Make it neat and tidy. Consider this a REPORT as well as a coding assignment. 

(\*) Last week you used the built in LinearRegression model to do SIMPLE linear regressrion. By default, the linear regression function uses least squares to estimate the coefficients, which means it works for multivariate linear regression too, by doing matrix math. 

So it's using a different estimation approach to acquire the same coefficients. You could print out THOSE coefficients and see if they are similar to the ones from SGD. There are OTHER ML models (most notably, any neural approach) where least squares DOES NOT work anymore, and we MUST use SGD (or some varient).

You can read more here:
* [SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html)
* [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
* [zeroR](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html)
* [Mean Squared Error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)



In [ ]:
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn import linear_model
import math
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Use this dataset for these examples

dataset = [[1,2,1],[2,2,3],[4,3,3],[3,2,2],[5,4,5],[5,2,6],[4,5,4],[7,5,5]]


# Create and shape our data
# **************************************************************************
# CREATE YOUR X_train,y_train, X_Test,y_test datasets here
# As before, you should use the same data for X_train and X_test
# AND y_train and y_test - although you can ALWAYS test your SGD against the 
# dataset I gave you earlier


X_train = []
for i in range(len(dataset)):
  X_train.append(dataset[i][:len(dataset[i])-1])

y_train = []
for i in range(len(dataset)):
  y_train.append(dataset[i][-1])

X_test = X_train
y_test = y_train


X_train = np.array(X_train).reshape(-1, 2)
y_train = np.array(y_train)

X_test = np.array(X_test).reshape(-1, 2)
y_test = np.array(y_test)

# Instantiate an object of the linear regression class
lr = LinearRegression()
reg = lr.fit(X_train,y_train)

# add linear regression
lr_predY = reg.predict(X_test)

# add ZeroR
zr = DummyRegressor()
zeroR = zr.fit(X_train, y_train)
zr_predY = zeroR.predict(X_test)

# **************************************************************************
# Create an instance of the SGD Regressor
# Create a model, by fitting the X_train values to the y_train values

sgd_clf = linear_model.SGDRegressor(max_iter=20)
sgd = sgd_clf.fit(X_train, y_train)

# Explore some built in attributes
# Note: even though we set max_iter to 200
# This implementation stops when the learning doesn't improve "enough"

print('INTERCEPT:',sgd.intercept_)
print('COEFFS:',sgd.coef_)
print('ITERATIONS TAKEN:',sgd.n_iter_)

# Test our model on the X_test data

sgd_predY = sgd.predict(X_test)

# Print out predicted vs. actual

print('\nPREDICTED : ACTUAL')
print('------------------')
for i in range(len(sgd_predY)):
  print('    {:.2f}  :  {:.2f}  '.format(sgd_predY[i],y_test[i]))
print()

#find RMSE for all three models
slr_rmse = math.sqrt(mean_squared_error(y_test,lr_predY))
zr_rmse = math.sqrt(mean_squared_error(y_test,zr_predY))
sgd_rmse = math.sqrt(mean_squared_error(y_test, sgd_predY))

#print rmse value of three
print('RMSE score for linear regression: {:.2f}'.format(slr_rmse))
print('RMSE score for zeroR: {:.2f}'.format(zr_rmse))
print('RMSE score for sgd_rmse: {:.2f}'.format(sgd_rmse))


INTERCEPT: [0.16735641]
COEFFS: [0.65908135 0.26950177]
ITERATIONS TAKEN: 20

PREDICTED : ACTUAL
------------------
    1.37  :  1.00  
    2.02  :  3.00  
    3.61  :  3.00  
    2.68  :  2.00  
    4.54  :  5.00  
    4.00  :  6.00  
    4.15  :  4.00  
    6.13  :  5.00  

RMSE score for linear regression: 0.83
RMSE score for zeroR: 1.58
RMSE score for sgd_rmse: 0.96


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


We have that Linear Regression is the best regarding the RMSE score. Since Linear Regression has the lowest RMSE among these three models. SGD is also okay since it's very close to Linear Regression but RMSE score is a bit higher. 